In [1]:
import os
import pandas as pd
import numpy as np
import time

import tushare as ts
ts.set_token('请填写你的token')
pro=ts.pro_api()

In [6]:
# 获取上一级路径
PATH = os.path.dirname(os.getcwd())
newdata_path = os.getcwd()+'/newdata'
# 读取量价数据
data = pd.read_feather(PATH+'/data/stk_daily.feather')
# 获取股票列表和交易日期列表
stock_list = data['stk_id'].unique().tolist()
trade_dates = data['date'].unique().tolist()
len(stock_list),len(trade_dates)

(5149, 728)

## 1 市值数据

In [3]:
# 初始化总市值和流通市值
total_mv = []; circ_mv = []
cnt = 0; tmp = 0

In [15]:
# 一分钟只能读200次，中断了就再运行
for stk in stock_list[cnt:]:
    try:
        # 获取股票数据
        df = pro.daily_basic(ts_code=stk, start_date='20200101', end_date='20221231').set_index('trade_date')
        # 转换日期格式
        df.index = pd.to_datetime(df.index)
        # 重新索引
        df = df[['total_mv','circ_mv']].reindex(trade_dates)
        # 保存数据
        total_mv.append(df['total_mv'].rename(stk))
        circ_mv.append(df['circ_mv'].rename(stk))
        # 记录进度
        tmp += 1
        # 跟踪进度
        if tmp%200 == 0:
            print(tmp,end=' Done! ')
            time.sleep(35)
    except:
        # tushare接口每分钟最多调取200次
        print(tmp,end=' Interrupt! ')
        cnt = tmp
        break

4000 Done! 4200 Done! 4400 Done! 4600 Done! 4800 Done! 5000 Done! 

In [18]:
len(total_mv),len(circ_mv)
# cnt

(5149, 5149)

In [19]:
total_mv = pd.concat(total_mv,axis=1)
circ_mv = pd.concat(circ_mv,axis=1)
display(total_mv.head()); display(circ_mv.head())

,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,872392.BJ,872808.BJ,872925.BJ,873001.BJ,873122.BJ,873169.BJ,873223.BJ,873305.BJ,873339.BJ,873527.BJ
trade_date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,3.273778e+07,3.679978e+07,187519.9354,332380.5684,728997.3248,330857.8820,1.031675e+06,1.717757e+06,279570.4571,566776.1165,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,3.333937e+07,3.622337e+07,187687.8887,333439.1052,739797.2852,329472.0898,1.042798e+06,1.650697e+06,279570.4571,565584.1583,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,3.312590e+07,3.561305e+07,182649.2877,333439.1052,737097.2951,328432.7457,1.037237e+06,1.815767e+06,281210.1666,559624.3674,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,3.328115e+07,3.589561e+07,185588.4716,338731.7894,741147.2803,329472.0898,1.053921e+06,1.846717e+06,290228.5684,569160.0329,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,3.233026e+07,3.580519e+07,182901.2178,333439.1052,727647.3298,325661.1614,1.034456e+06,1.833821e+06,287769.0043,553664.5765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,872392.BJ,872808.BJ,872925.BJ,873001.BJ,873122.BJ,873169.BJ,873223.BJ,873305.BJ,873339.BJ,873527.BJ
trade_date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,3.273750e+07,3.163259e+07,184456.0361,332195.1200,728086.3049,295045.3820,962288.9957,1.694195e+06,178033.8828,500646.8831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,3.333908e+07,3.113712e+07,184621.2452,333253.0662,738872.7687,293809.5898,972664.0793,1.628056e+06,178033.8828,499593.9980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06,3.312562e+07,3.061250e+07,179664.9702,333253.0662,736176.1527,292882.7457,967476.5375,1.790861e+06,179078.0698,494329.5723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-07,3.328087e+07,3.085538e+07,182556.1306,338542.7974,740221.0766,293809.5898,983039.1628,1.821387e+06,184821.0983,502752.6534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-08,3.232998e+07,3.077766e+07,179912.7840,333253.0662,726737.9969,290411.1614,964882.7666,1.808668e+06,183254.8178,489065.1466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
total_mv.to_feather(PATH+'/newdata/tot_mv.feather')
circ_mv.to_feather(PATH+'/newdata/circ_mv.feather')

## 2 行业数据

In [88]:
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
data.to_feather(PATH+'/newdata/stock_basic.feather')

## 3 指数数据

In [7]:
dt = pro.index_basic()
dt.to_feather(newdata_path+'/index_basic.feather')
dt

,ts_code,name,market,publisher,category,base_date,base_point,list_date
0,000001.CJ,能源(长江),OTH,长江证券股份有限公司,行业指数,20021231,1000.00,20200910
1,000001.SH,上证指数,SSE,中证指数有限公司,综合指数,19901219,100.00,19910715
2,000001CNY01.SH,上证指数全收益,SSE,中证指数有限公司,综合指数,20200721,3320.89,20201009
3,000002.CJ,原材料(长江),OTH,长江证券股份有限公司,行业指数,20021231,1000.00,20200910
4,000002.SH,A股指数,SSE,中证指数有限公司,规模指数,19901219,100.00,19920221
...,...,...,...,...,...,...,...,...
7995,983085.CNI,深港通节能,CNI,深圳证券交易所,主题指数,20111230,1000.00,20180308
7996,983086.CNI,深港通节能(港币),CNI,深圳证券交易所,主题指数,20111230,1000.00,20180308
7997,983087.CNI,深港通新能源车,CNI,深圳证券交易所,主题指数,20111230,1000.00,20180308
7998,983088.CNI,深港通新能源车(港币),CNI,深圳证券交易所,主题指数,20111230,1000.00,20180308


In [9]:
dt_SSE = dt[dt['market']=='SSE']
len(dt_SSE)

310

In [103]:
index_close = []
for i,idx in enumerate(dt_SSE.ts_code):
    # 跟踪进度
    if (i+1)%100 == 0:
        print(i+1,end=' Done! ')
    # 获取指数数据
    df = pro.index_daily(ts_code=idx, start_date='20200101', end_date='20221231').set_index('trade_date')
    # 转换日期格式
    df.index = pd.to_datetime(df.index)
    index_close.append(df['close'].rename(idx))
index_close = pd.concat(index_close,axis=1)

200 Done! 

In [105]:
index_close = index_close.dropna(how='all',axis=1)
index_close = index_close.dropna(how='all',axis=0)
index_close

,000001.SH,000002.SH,000003.SH,000004.SH,000005.SH,000006.SH,000007.SH,000008.SH,000009.SH,000010.SH,...,000988.SH,000989.SH,000991.SH,000992.SH,000993.SH,000994.SH,000995.SH,931845.SH,931944.SH,950092.SH
trade_date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,3085.1976,3232.6892,261.1236,2420.2994,2922.3713,7174.6260,4997.0545,2999.1744,4881.7235,8970.5173,...,2948.6987,4688.1787,10011.7326,6327.4838,6064.0086,6153.7905,2014.9438,NaN,NaN,5271.4229
2020-01-03,3083.7858,3231.1885,261.7619,2415.5837,2914.4835,7152.1163,4988.3279,3006.5318,4879.5471,8946.5499,...,2958.7467,4687.3190,9985.6603,6326.8923,6104.9657,6187.8268,2017.9545,NaN,NaN,5259.3481
2020-01-06,3083.4083,3230.7984,261.5396,2424.6302,2909.1061,7143.3927,4974.0985,2993.5497,4901.7366,8903.6367,...,2968.2121,4695.3318,9936.5227,6287.6033,6185.2001,6223.6771,2019.4824,NaN,NaN,5225.2272
2020-01-07,3104.8015,3253.1915,264.1561,2439.3916,2941.9118,7165.1176,5004.6836,3016.3291,4957.3759,8957.6528,...,2993.1193,4780.0711,10093.4872,6327.7491,6254.9061,6256.4957,2035.0780,NaN,NaN,5236.1678
2020-01-08,3066.8925,3213.4326,262.2829,2422.2487,2879.4709,7048.6963,4939.7350,2965.2891,4888.8867,8844.5560,...,2970.8885,4772.9172,9977.3290,6198.0531,6148.2935,6130.9702,2009.4265,NaN,NaN,5165.1208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,3065.5626,3213.2804,278.2372,2814.6578,2490.6827,6014.8839,4388.7153,2479.2099,5544.8308,8148.7924,...,3840.5334,4718.5199,10763.6769,4627.1671,4954.2637,4632.8049,2211.6466,NaN,NaN,4820.6744
2022-12-27,3095.5678,3244.7106,281.4323,2838.7088,2523.1523,6052.5982,4417.1017,2511.3776,5606.1400,8233.5199,...,3879.5822,4753.7461,10741.1667,4698.8000,4980.2666,4651.4292,2221.3043,NaN,NaN,4840.8815
2022-12-28,3087.3997,3236.1804,280.5334,2824.9520,2518.7301,6060.1806,4405.5295,2516.3872,5578.9870,8232.7204,...,3838.6794,4672.4767,10772.5789,4721.3442,4910.8439,4614.5970,2273.2396,NaN,NaN,4883.0376


In [106]:
index_close.to_feather(PATH+'/newdata/index_close.feather')